## Lab | Inferential statistics - T-test & P-value

Libraries.

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

#### Exercise 1: One tailed t-test

In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file files_for_lab/machine.txt. Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

In [16]:
machine = pd.read_csv('files_for_lab/machine.txt', sep='\t', encoding='utf-16')
print('Shape:', machine.shape)
machine.head()

Shape: (10, 2)


,New machine,Old machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5


We will compare **two independent samples** (because the new machine performance is not related to the old machine performance). Our hypothesis is the following:

$$H0: \mu_{secondsnew} < \mu_{secondsold}$$
$$H1: \mu_{secondsnew} ≥ \mu_{secondsold}$$

The new machine will pack faster than the old one, that is: it will take less time (fewer seconds) for the new machine to pack cartons with jars than the old machine. 

* We will set the significance level at **0.05.**
* The region of rejection will be on the **right-side.**
* Our sample will be the whole data frame (that contains 10 rows).

In [43]:
sample = machine
sample.head()
sample = sample.rename(columns = {'    Old machine': 'Old machine'})

In [44]:
sample_new, sample_old = sample['New machine'], sample['Old machine']

In [45]:
sample_new_mean, sample_old_mean = sample_new.mean(), sample_old.mean()
sample_new_std, sample_old_std = sample_new.std(ddof=1), sample_old.std(ddof=1)
sp = ( len(sample_new) - 1 ) * ( sample_new_std**2 ) +  ( len(sample_old) - 1 ) * ( sample_old_std**2 )
sp /= ( len(sample_new) + len(sample_old) - 2)
r = np.sqrt( (1/len(sample_new)) + (1/len(sample_old)) )
t = (sample_new_mean - sample_old_mean)/ (sp * r)

print("The mean of sample new is {:.2f}".format(sample_new_mean))
print("The mean of sample old is {:.2f}".format(sample_old_mean))
print("The standard deviation of sample new is {:.2f}".format(sample_new_std))
print("The standard deviation of sample old is {:.2f}".format(sample_old_std))
print("The ratio of the sample variances is {:.2f} which is bigger than 0.5 and smaller than 2".format(sample_new_std/sample_old_std))
print("The t statistic is: {:.2f}".format(t))

The mean of sample new is 42.14
The mean of sample old is 43.23
The standard deviation of sample new is 0.68
The standard deviation of sample old is 0.75
The ratio of the sample variances is 0.91 which is bigger than 0.5 and smaller than 2
The t statistic is: -4.74


Calculate critical value and p-value:

In [65]:
print('Critical value:', round(st.t.ppf(1-(0.05),df = len(sample_new)+len(sample_old)-2),3))
print('P-value:', round(st.t.cdf(t,df = len(sample_new)+len(sample_old)-2),6))

Critical value: 1.734
P-value: 0.021169


According to these results, we'll **reject the null hypothesis.**

# ................................................................................................. # 

#### Exercise 2: Matched Pairs Test
In this challenge we will compare dependent samples of data describing our Pokemon (file files_for_lab/pokemon.csv). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment your result.

In [33]:
pokemon = pd.read_csv('files_for_lab/pokemon.csv')
print('Shape:', pokemon.shape)
pokemon.head()

Shape: (800, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In this test, we will assume that the data in the two samples are dependent. The columns we'll compare are 'Attack' and 'Defense'. Our hypothesis is:

$$H0: \mu_{attack} = \mu_{defense}$$
$$H1: \mu_{attack} \neq \mu_{defense}$$

This is a two-sided test, therefore the rejection area will be located on **both sides.**

* The significance level will be **0.05.**
* We will pick a sample of **30 items.**

In [35]:
pokemon_sample = pokemon[['Attack', 'Defense']].sample(30)
pokemon_sample['difference'] = pokemon_sample['Defense']-pokemon_sample['Attack']
pokemon_sample.head()

,Attack,Defense,difference
191,50,45,-5
654,60,70,10
750,150,50,-100
407,95,100,5
349,140,70,-70


In [57]:
pokemon_sample_mean, pokemon_sample_std = pokemon_sample['difference'].mean(), pokemon_sample['difference'].std(ddof=1)
print('Mean:', round(pokemon_sample_mean,2), '\nStd:', round(pokemon_sample_std,2))

Mean: -15.37 
Std: 38.52


In [56]:
t = pokemon_sample_mean / ( pokemon_sample_std / np.sqrt(pokemon_sample.shape[0]) )
print("The mean of our samples differences is: {:.2f}".format(pokemon_sample_mean))
print("The standard deviation of our samples differences is: {:.2f}".format(pokemon_sample_std))
print("Our t statistics is: {:.2f}".format(t))

The mean of our samples differences is: -15.37
The standard deviation of our samples differences is: 38.52
Our t statistics is: -2.19


In [71]:
tc = st.t.ppf(1-(0.05/2),df= pokemon_sample.shape[0] - 1)
print('Critical value:', '+',round(tc,3), '/ -',round(tc,3))

Critical value: + 2.045 / - 2.045


Our statistic is -2.19 while the critical values are +2.045/-2.045. Then, as -2.19 < -2.045 **we should reject the H0.** Let's check the p-value.

In [69]:
print('P-value:', round(st.t.cdf(t,df = pokemon_sample.shape[0] - 1),3))

P-value: 0.019


In [42]:
st.ttest_rel(pokemon_sample['Defense'],pokemon_sample['Attack'])

Ttest_relResult(statistic=-2.1851895751143076, pvalue=0.037100457353611976)

Again, P-value is lower than 0.05, therefore we **reject the HO.**

# ................................................................................................. # 

#### Exercise 3
In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on: 
- Null hypothesis 
- Alternate hypothesis 
- Level of significance 
- Test statistic 
- P-value 
- F table

##### Context

Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data anova_lab_data.xlsx file in the files_for_lab folder

* State the null hypothesis
* State the alternate hypothesis
* What is the significance level
* What are the degrees of freedom of model, error terms, and total DoF

Data was collected randomly and provided to you in the table as shown: https://education-team-2020.s3-eu-west-1.amazonaws.com/data-analytics/7.05/7.05-lab_data.png

### Solution

#### Hypothesis

ANOVA test takes samples of more than two different populations and test if all the sample means are the same or not (no matter which pair of means is different). Therefore, our hypothesis is:

$$H0: \mu_{1}=\mu_{2}=\mu_{3}=⋯=\mu_{n}$$
$$H1: \mu_{i}\neq\mu_{j}$$

for any i,j

#### Significance level
The significance level will be 0.05.

#### Degrees of freedom

In [9]:
pbeam = pd.read_excel('files_for_lab/anova_lab_data.xlsx')
print('Shape:', pbeam.shape)
pbeam.head()

Shape: (15, 2)


,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71


In [10]:
pbeam = pbeam.rename(columns={'Power ': 'Power'})
pbeam.columns

Index(['Power', 'Etching Rate'], dtype='object')

In [11]:
print('N is:', len(pbeam))
print('k is:', len(pbeam['Power'].unique()))

N is: 15
k is: 3


Therefore, degrees of freedom are:
* Total: N-1 / 15-1 = 14
* Between samples: k-1 / 3-1 = 2
* Within samples: n-k / 15 - 3 = 12

#### In this section, use the Python to conduct ANOVA. What conclusions can you draw from the experiment and why?

In [12]:
group_df = pbeam.groupby('Power')['Etching Rate'].agg(etching_mean='mean', Samples='size').reset_index()
group_df

,Power,etching_mean,Samples
0,160 W,5.792,5
1,180 W,6.238,5
2,200 W,8.318,5


Calculate SST:

In [13]:
S2t = 0
for v in pbeam['Power'].unique():
    ng = len(pbeam[pbeam['Power'] == v])  
    S2t  += ( ( pbeam[pbeam['Power'] == v]['Etching Rate'].mean() - pbeam['Etching Rate'].mean() ) ** 2) * ng
S2t /= ( pbeam['Power'].nunique() - 1 )
print("The value of S2t is {:.2f}".format(S2t)) 

The value of S2t is 9.09


Calculate SSE:

In [14]:
S2E = 0
for v in pbeam['Power'].unique():
    for er in pbeam[pbeam['Power'] == v]['Etching Rate']:
        S2E += ( er - pbeam[pbeam['Power'] == v]['Etching Rate'].mean() ) ** 2
S2E /= ( len(pbeam) - pbeam['Power'].nunique() ) 

print()
print("The value of S2E is {:.2f}".format(S2E))


The value of S2E is 0.25


In [15]:
F = S2t / S2E
print("The value of F is {:.2f}".format(F))

The value of F is 36.88


Let's double check the results with f_oneway:

In [17]:
st.f_oneway(pbeam[pbeam['Power'] == '160 W']['Etching Rate'],
            pbeam[pbeam['Power'] == '180 W']['Etching Rate'],
            pbeam[pbeam['Power'] == '200 W']['Etching Rate'])

F_onewayResult(statistic=36.87895470100505, pvalue=7.506584272358903e-06)

**Conclusion:**

As p-value is lower than 0.05, we **reject the null hypothesis** and conclude that the samples **are not equal**.